In [6]:
import os
import pinecone

import custom_loaders

In [8]:
# complete rewrite of langchain git loader using parallelism
loader = custom_loaders.TurboGitLoader(
    clone_url="https://github.com/hwchase17/langchain",
    repo_path="./example_data/TurboGitLoader/",
    branch="master",
    file_filter=lambda file_path: file_path.endswith(".py")
)
data = loader.load()
print(f"Length of data from dataloader: {len(data)}")

Length of data from dataloader: 937


In [18]:
dict(data[0])

{'page_content': '"""Prompt for the router chain in the multi-prompt chain."""\r\n\r\nMULTI_PROMPT_ROUTER_TEMPLATE = """\\\r\nGiven a raw text input to a language model select the model prompt best suited for \\\r\nthe input. You will be given the names of the available prompts and a description of \\\r\nwhat the prompt is best suited for. You may also revise the original input if you \\\r\nthink that revising it will ultimately lead to a better response from the language \\\r\nmodel.\r\n\r\n<< FORMATTING >>\r\nReturn a markdown code snippet with a JSON object formatted to look like:\r\n```json\r\n{{{{\r\n    "destination": string \\\\ name of the prompt to use or "DEFAULT"\r\n    "next_inputs": string \\\\ a potentially modified version of the original input\r\n}}}}\r\n```\r\n\r\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR \\\r\nit can be "DEFAULT" if the input is not well suited for any of the candidate prompts.\r\nREMEMBER: "next_inputs" can 

In [20]:
import tiktoken

tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [21]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [77]:
from langchain.embeddings.openai import OpenAIEmbeddings
from getpass import getpass

model_name = 'text-embedding-ada-002'
OPENAI_API_KEY = getpass('OPENAI_API_KEY=')
embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY,
)

In [54]:
data[2].page_content

'"""An agent designed to hold a conversation in addition to using tools."""\r\n'

In [55]:
embed_resp = embed.embed_documents(data[2].page_content)
len(embed_resp), len(embed_resp[0])

(76, 1536)

In [68]:
import pinecone

index_name = 'testindex'
pinecone.init(
    api_key=getpass('PINECONE_API_KEY'), 
    environment="us-west1-gcp-free"
)

if index_name in pinecone.list_indexes():
    print(f"Index {index_name} already exists...")
else:
    pinecone.create_index(
        name=index_name,
        metric='dotproduct',
        dimension=len(embed_resp[0])
)

Index testindex already exists...


In [70]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [72]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 100

texts = []
metadatas = []

for i, record in enumerate(tqdm(data)):
    # first get metadata fields for this record
    metadata = {
        'file_name': record.metadata['file_name'],
        'file_path': record.metadata['file_path']
    }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(record.page_content)
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "text": text, **metadata
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []

if len(texts) > 0:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))

  0%|          | 0/937 [00:00<?, ?it/s]

In [73]:

index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 2539}},
 'total_vector_count': 2539}

In [74]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [75]:
query = "What are the main components of Langchain?"

vectorstore.similarity_search(
    query,
    k=3
)

[Document(page_content='"""Chains are easily reusable components which can be linked together."""\r\nfrom langchain.chains.api.base import APIChain\r\nfrom langchain.chains.api.openapi.chain import OpenAPIEndpointChain\r\nfrom langchain.chains.combine_documents.base import AnalyzeDocumentChain\r\nfrom langchain.chains.constitutional_ai.base import ConstitutionalChain\r\nfrom langchain.chains.conversation.base import ConversationChain\r\nfrom langchain.chains.conversational_retrieval.base import (\r\n    ChatVectorDBChain,\r\n    ConversationalRetrievalChain,\r\n)\r\nfrom langchain.chains.flare.base import FlareChain\r\nfrom langchain.chains.graph_qa.base import GraphQAChain\r\nfrom langchain.chains.hyde.base import HypotheticalDocumentEmbedder\r\nfrom langchain.chains.llm import LLMChain\r\nfrom langchain.chains.llm_bash.base import LLMBashChain\r\nfrom langchain.chains.llm_checker.base import LLMCheckerChain\r\nfrom langchain.chains.llm_math.base import LLMMathChain\r\nfrom langchain.

In [78]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [79]:
qa.run(query)


'Langchain has several components, including chains, agents, llms, prompts, docstore, sql_database, and utilities. Some of the chains include ConversationChain, LLMBashChain, LLMChain, LLMCheckerChain, LLMMathChain, PALChain, QAWithSourcesChain, SQLDatabaseChain, SequentialChain, SimpleSequentialChain, VectorDBQA, VectorDBQAWithSourcesChain, APIChain, LLMRequestsChain, TransformChain, MapReduceChain, OpenAIModerationChain, SQLDatabaseSequentialChain, load_chain, AnalyzeDocumentChain, HypotheticalDocumentEmbedder, ChatVectorDBChain, GraphQAChain, ConstitutionalChain, QAGenerationChain, RetrievalQA, RetrievalQAWithSourcesChain, ConversationalRetrievalChain, OpenAPIEndpointChain, and FlareChain.'

In [80]:
pinecone.delete_index("testindex")
